In [53]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics.classification import BinaryConfusionMatrix

import tqdm

import copy

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix, classification_report

import torch.utils.data as data_utils

In [2]:
device = torch.device("cuda:0")

## Read in data 

In [30]:
#train = np.load('/content/baseline_train_smote.npz')
train = np.load("/content/baseline_train.npz")

In [32]:
train["x"].shape

(756075, 106)

In [31]:
train["y"].shape

(756075,)

In [33]:
#validation = np.load('/content/baseline_validation_smote.npz')
validation = np.load('/content/baseline_validation.npz')

In [34]:
validation["x"].shape

(161492, 106)

In [35]:
validation["y"].shape

(161492,)

In [36]:
#test = np.load("/content/baseline_test_smote.npz")
test = np.load("/content/baseline_test.npz")

In [37]:
test["x"].shape 

(161924, 106)

In [38]:
test["y"].shape

(161924,)

In [54]:
# Load data 
minibatch_size = 500

X_train = torch.tensor(train["x"], dtype=torch.float32).to(device)
y_train = torch.tensor(train["y"], dtype=torch.float32).reshape(-1,1).to(device)
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=minibatch_size, shuffle=True)

X_val = torch.tensor(validation["x"], dtype=torch.float32).to(device)
y_val = torch.tensor(validation["y"], dtype=torch.float32).reshape(-1,1).to(device)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=validation["x"].shape[0], shuffle=False)

X_test = torch.tensor(test["x"], dtype=torch.float32).to(device)
y_test = torch.tensor(test["y"], dtype=torch.float32).reshape(-1,1).to(device)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

## Model setup 

In [45]:
class FF1(nn.Module):
    def __init__(self, input_size, hl1, hl2):
        super().__init__()
        self.hl1 = nn.Linear(input_size, hl1)
        self.hl2 = nn.Linear(hl1, hl2)
        self.output = nn.Linear(hl2, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.hl1(x))
        x = self.relu(self.hl2(x))
        x = self.sigmoid(self.output(x))

        return x

In [46]:
# Initialize model 
input_size = 106
hl1 = 66 
hl2 = 44
curr_model = FF1(input_size, hl1, hl2)
curr_model = curr_model.to(device)

# Initialize Hyperparameters
learning_rate = 0.001
num_epochs = 10
criterion = nn.BCELoss()
#criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(curr_model.parameters(), 
                             lr=learning_rate, 
                             weight_decay=10e-05)

# To store loss for plotting 
history = {}
history['train_loss'] = []
history['validation_loss'] = []
history['validation_true_label'] = []
history['test_true_label'] = []

# To store accuracy
accuracy = {}
accuracy['validation_accuracy'] = []
accuracy['test_accuracy'] = []

In [42]:
print(sum([x.reshape(-1).shape[0] for x in curr_model.parameters()])) #number of parameters 

10055


In [47]:
for epoch in range(num_epochs):
  h = np.array([])
  for data, label in train_loader:
    # move data and label to device
    data = data.to(device)
    label = label.to(device)
    
    # forward
    output = curr_model(data)
    loss = criterion(output, label)
    h = np.append(h, loss.item())

    # backward 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Record loss values 
  mean_loss = np.mean(h)
  print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, mean_loss))
  history['train_loss'].append(mean_loss)

epoch [1/10], loss:0.0146
epoch [2/10], loss:0.0027
epoch [3/10], loss:0.0027
epoch [4/10], loss:0.0027
epoch [5/10], loss:0.0027
epoch [6/10], loss:0.0027
epoch [7/10], loss:0.0027
epoch [8/10], loss:0.0027
epoch [9/10], loss:0.0027
epoch [10/10], loss:0.0027


In [61]:
# Evaluate accuracy on validation data 
curr_model.eval()

with torch.no_grad():
    for data, label in val_loader:
        # move data and label to device
        data = data.to(device)
        label = label.to(device)

        output = curr_model(data).to(device)
        #print(f"Output: {output}")

        # loss
        loss = criterion(output, label)
        print(f"Loss: {loss}")
        history['validation_loss'].append(loss.item())
        history['validation_true_label'].append(label)

        # accuracy
        acc = (output.round() == label).float().mean()
        acc = float(acc)
        print(f"Accuracy: {acc}")
        accuracy["validation_accuracy"].append(acc) 

Loss: 0.002571320394054055
Accuracy: 0.9996966123580933


In [63]:
bcm = BinaryConfusionMatrix().to(device)
bcm(output, label)

tensor([[161443,      0],
        [    49,      0]], device='cuda:0')

In [50]:
np.mean(history["validation_loss"])

0.002571217041936971